In [7]:
import tqdm
from modelscope import AutoModelForCausalLM, AutoTokenizer
model_name = 'Qwen/Qwen2.5-0.5B-Instruct'
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

2025-10-01 14:42:12,215 - modelscope - INFO - Target directory already exists, skipping creation.


2025-10-01 14:42:14,390 - modelscope - INFO - Target directory already exists, skipping creation.


In [8]:
from datasets import load_dataset
data = load_dataset('gsm8k', 'main')

In [9]:
def gen(question):
    prompt = 'According to the following questions, please give detailed reasoning steps and answers in the following format: \n<reasoning>\n{reasoning}\n</reasoning>\n<answer>\n{answer}\n</answer>' + f"\n{question}\n"
    messages = [
        {"role": "user", "content": prompt}
        ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True, # add new response prompt at the end
        )
    model_inputs = tokenizer(
        [text],
        return_tensors="pt",
        ).to(model.device)
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512,
        )
    
    return tokenizer.batch_decode(
        generated_ids,
        skip_special_tokens=False
        )[0]

In [13]:
data['test'][0]

{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
 'answer': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18'}

In [17]:
a = gen(data['test'][0]['question'])
print(a)

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
According to the following questions, please give detailed reasoning steps and answers in the following format: 
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
<|im_end|>
<|im_start|>assistant
Let's break down Janet's daily earnings from selling the remaining eggs:

1. Breakfast:
   - Janet eats 3 eggs each morning.
   - Therefore, she has 16 total breakfast eggs minus her daily egg consumption of 3.

2. Baking muffins:
   - She bakes muffins for her friends every day with 4 muffins.
   - So, there are no additional eggs left after breakfast.

3. Selling remaining eggs:
   - Afte